In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import sys
import Queue
import threading
import math


In [2]:

try:
    sys.path.append(os.path.join(os.environ.get("SUMO_HOME",os.path.join(os.getcwd(), '..')), "tools"))
    from sumolib import checkBinary  # noqa
    import traci
except ImportError:
    sys.exit(
        "please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")


In [3]:
from Tkinter import *

eventQueue = Queue.Queue()
TS = 0.05
VERBOSE = False


def downKey(event):
    eventQueue.put('slow')
    if VERBOSE:
        print("Down key pressed")


def upKey(event):
    eventQueue.put('fast')
    if VERBOSE:
        print("Up key pressed")


def leftKey(event):
    eventQueue.put('up')
    if VERBOSE:
        print("Left key pressed")


def rightKey(event):
    eventQueue.put('down')
    if VERBOSE:
        print("Right key pressed")

def nextCar(event):
    eventQueue.put('next_car')
    if VERBOSE:
        print("N key pressed")

def prevCar(event):
    eventQueue.put('prev_car')
    if VERBOSE:
        print("P key pressed")

In [4]:

class AdoVehicleControl:

    """
    Launch the main part of the GUI and the worker thread. periodicCall and
    endApplication could reside in the GUI part, but putting them here
    means that you have all the thread controls in a single place.
    """

    def __init__(self, master, sumocfg, egoID):
        self.master = master
        self.sumocfg = sumocfg
        self.egoID = egoID
        self.running = True

        self.thread = threading.Thread(target=self.workerThread)
        self.thread.start()
        self.type = "fast"
        # Start the periodic call in the GUI to see if it can be closed
        self.periodicCall()

    def periodicCall(self):
        if not self.running:
            sys.exit(1)
        self.master.after(100, self.periodicCall)

    def workerThread(self):
        try:
            traci.start([checkBinary("sumo-gui"), "-c", self.sumocfg,
                         # "--lateral-resolution", "0.05",
                         # "--collision.action", "warn",
                         # "--step-length", str(TS)
                         ])
            # steal focus for keyboard input after sumo-gui has loaded
            # self.master.focus_force() # not working on all platforms
            # make sure ego vehicle is loaded
            traci.simulationStep()
            speed = traci.vehicle.getSpeed(self.egoID)
            angle = traci.vehicle.getAngle(self.egoID)
            traci.vehicle.setSpeedMode(self.egoID, 0)
            steerAngle = 0
            x, y = traci.vehicle.getPosition(self.egoID)
            traci.gui.trackVehicle(traci.gui.DEFAULT_VIEW, self.egoID)
            while traci.simulation.getMinExpectedNumber() > 0:
                try:
                    if eventQueue.qsize():
                        button = eventQueue.get(0)
                        if button == "next_car":
                            p = self.egoID.split('.')
                            self.egoID = p[0] + '.' + str(int(p[1])+1)
                            speed = traci.vehicle.getSpeed(self.egoID)
                            angle = traci.vehicle.getAngle(self.egoID)
                            traci.vehicle.setSpeedMode(self.egoID, 0)
                            steerAngle = 0
                            x, y = traci.vehicle.getPosition(self.egoID)
                            traci.gui.trackVehicle(traci.gui.DEFAULT_VIEW, self.egoID)
                            continue
                        elif button == "prev_car":
                            p = self.egoID.split('.')
                            self.egoID = p[0] + '.' + str(int(p[1])-1)
                            speed = traci.vehicle.getSpeed(self.egoID)
                            angle = traci.vehicle.getAngle(self.egoID)
                            traci.vehicle.setSpeedMode(self.egoID, 0)
                            steerAngle = 0
                            x, y = traci.vehicle.getPosition(self.egoID)
                            traci.gui.trackVehicle(traci.gui.DEFAULT_VIEW, self.egoID)
                            continue
                        else:
                            self.type = button
                            if self.type == "fast":
                                speed += 2
                            elif self.type == "slow":
                                speed -= 2
                            elif self.type == "up":
                                angle -= 5
                            elif self.type == "down":
                                angle += 5
                            else:
                                print("WHAT ARE YOU DOING?!?!")
                except Queue.Empty:
                    pass
                # angle += steerAngle
                angle = angle % 360
                rad = angle / 180 * math.pi + 0.5 * math.pi
                x2 = x - math.cos(rad) * TS * speed
                y2 = y + math.sin(rad) * TS * speed
                traci.vehicle.setSpeed(self.egoID, speed)
                traci.vehicle.moveToXY(self.egoID, "1", 0, x2, y2, angle, keepRoute=2)
                x3, y3 = traci.vehicle.getPosition(self.egoID)
                x, y = x2, y2
                traci.simulationStep()
                if VERBOSE:
                    print("old=%.2f,%.2f new=%.2f,%.2f found=%.2f,%.2f speed=%.2f steer=%.2f angle=%s rad/pi=%.2f cos=%.2f sin=%.2f" % (
                        x, y, x2, y2, x3, y3, speed, steerAngle, angle, rad / math.pi,
                        math.cos(rad), math.sin(rad)))
            traci.close()
        except traci.FatalTraCIError:
            pass
        self.running = False



In [5]:

def main(sumocfg="/home/karen/projects/trams/explanatory_factors/sumo-files/lanechange_fancy.sumo.cfg", egoID="exit.0"):
    root = Tk()
    root.geometry('180x100+0+0')
    frame = Frame(root)
    Button(frame, text="Click here.\nControl with arrow keys").grid(row=0)
    root.bind('<Left>', leftKey)
    root.bind('<Right>', rightKey)
    root.bind('<Up>', upKey)
    root.bind('<Down>', downKey)
    root.bind('n', nextCar)
    root.bind('p', prevCar)
    frame.pack()

    client = AdoVehicleControl(root, sumocfg, egoID)
    root.mainloop()

if len(sys.argv) < 3:
    main(*sys.argv[1:])
else:
    print("racing.py <sumocfg> [<egoID>]")


racing.py <sumocfg> [<egoID>]


In [8]:
main()


Could not connect to TraCI server at localhost:38887 [Errno 111] Connection refused
 Retrying in 1 seconds


KeyboardInterrupt: 